# Chatbot for E-commerce

## 1. Data Analysis

In [37]:
import json
import nltk

words=[]
classes = []
documents = []

ignore_words = ['?', '!', ".", ",", "'s"]

data_file = open('intents.json').read()
intents = json.loads(data_file)

In [38]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, I am your Online Shopping Assistant Bots.',
    'Hello, Good to see you.',
    'Hi there, how can I help you?.'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time',
    'Well i am finished'],
   'responses': ['See you! Thanks for shopping with us.',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'buybooks',
   'patterns': ['I want to buy a romantic comedy genre book',
    'I want to buy the newest novel you have',
    'Do you sell books?',
    

In [39]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [40]:
print("Classes: ", classes, "\n")
print("Words: ", words, "\n")
print("Documents: ", documents, "\n")

Classes:  ['greeting', 'goodbye', 'thanks', 'buybooks', 'payment', 'noanswer'] 

Words:  ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Well', 'i', 'am', 'finished', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'I', 'want', 'to', 'buy', 'a', 'romantic', 'comedy', 'genre', 'book', 'I', 'want', 'to', 'buy', 'the', 'newest', 'novel', 'you', 'have', 'Do', 'you', 'sell', 'books', '?', 'Can', 'you', 'recommend', 'me', 'some', 'books', '?', 'How', 'can', 'i', 'pay', '?', 'How', 'much', 'money', 'does', 'this', 'cost', '?', 'Check', 'out', 'please', 'I', 'am', 'finished', 'shopping', '.'] 

Documents:  [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), 

## 2. Preprocess data


In [41]:
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
import random
import numpy as np

lemmatizer = WordNetLemmatizer()

In [42]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print(words)

['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'me', 'money', 'much', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'well', 'you']


In [43]:
# sort classes
classes = sorted(list(set(classes)))
classes

['buybooks', 'goodbye', 'greeting', 'noanswer', 'payment', 'thanks']

In [44]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


60 unique lemmatized words ['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'me', 'money', 'much', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'well', 'you']


In [45]:
with open("words_data.json", "w") as words_data:
    json.dump(words, words_data)

In [46]:
with open("classes_data.json", "w") as classes_data:
    json.dump(classes, classes_data)

## 3. Create training and testing data


In [47]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [49]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [52]:
for t in training:
    print(t)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0

In [53]:
output_empty

[0, 0, 0, 0, 0, 0]

In [54]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])
print("Training data created")

Training data created


In [55]:
y_train

[[0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0,

In [56]:
for i in range(len(X_train)):
    print(X_train[i], y_train[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] [1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1] [0, 1, 0, 0, 0, 0]


## 4.1 Build the Neural Network Model

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [58]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [59]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [60]:
#fitting and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
11/11 [==============================] - 0s 1ms/step - loss: 1.7965 - accuracy: 0.2407
Epoch 2/200
11/11 [==============================] - 0s 907us/step - loss: 1.6785 - accuracy: 0.2778
Epoch 3/200
11/11 [==============================] - 0s 810us/step - loss: 1.5560 - accuracy: 0.4259
Epoch 4/200
11/11 [==============================] - 0s 725us/step - loss: 1.4513 - accuracy: 0.5556
Epoch 5/200
11/11 [==============================] - 0s 813us/step - loss: 1.3382 - accuracy: 0.5741
Epoch 6/200
11/11 [==============================] - 0s 1ms/step - loss: 1.2543 - accuracy: 0.6296
Epoch 7/200
11/11 [==============================] - 0s 999us/step - loss: 1.0766 - accuracy: 0.7407
Epoch 8/200
11/11 [==============================] - 0s 1ms/step - loss: 1.0247 - accuracy: 0.6667
Epoch 9/200
11/11 [==============================] - 0s 1ms/step - loss: 0.9425 - accuracy: 0.7407
Epoch 10/200
11/11 [==============================] - 0s 1ms/step - loss: 0.8098 - accuracy: 0.8519

11/11 [==============================] - 0s 550us/step - loss: 0.0310 - accuracy: 1.0000
Epoch 82/200
11/11 [==============================] - 0s 635us/step - loss: 0.0279 - accuracy: 1.0000
Epoch 83/200
11/11 [==============================] - 0s 934us/step - loss: 0.0508 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 613us/step - loss: 0.0213 - accuracy: 1.0000
Epoch 85/200
11/11 [==============================] - 0s 725us/step - loss: 0.0302 - accuracy: 1.0000
Epoch 86/200
11/11 [==============================] - 0s 907us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 87/200
11/11 [==============================] - 0s 544us/step - loss: 0.0236 - accuracy: 1.0000
Epoch 88/200
11/11 [==============================] - 0s 689us/step - loss: 0.0902 - accuracy: 0.9815
Epoch 89/200
11/11 [==============================] - 0s 876us/step - loss: 0.0344 - accuracy: 0.9815
Epoch 90/200
11/11 [==============================] - 0s 725us/step - loss: 0.0202 - accuracy: 

Epoch 161/200
11/11 [==============================] - 0s 852us/step - loss: 0.0391 - accuracy: 0.9815
Epoch 162/200
11/11 [==============================] - 0s 673us/step - loss: 0.0102 - accuracy: 1.0000
Epoch 163/200
11/11 [==============================] - 0s 725us/step - loss: 0.0529 - accuracy: 0.9815
Epoch 164/200
11/11 [==============================] - 0s 819us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 638us/step - loss: 0.0364 - accuracy: 0.9815
Epoch 166/200
11/11 [==============================] - 0s 790us/step - loss: 0.0074 - accuracy: 1.0000
Epoch 167/200
11/11 [==============================] - 0s 635us/step - loss: 0.0118 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 622us/step - loss: 0.0397 - accuracy: 0.9815
Epoch 169/200
11/11 [==============================] - 0s 725us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 170/200
11/11 [==============================] - 0s 717us/step - lo